In [1]:
!git clone https://github.com/vgupta123/sumpubmed

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.


In [2]:
%cd sumpubmed

/content/sumpubmed


In [3]:
!pip install datasets
!pip install pandas
!pip install re
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [4]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
from datasets import Dataset
import pandas as pd
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline

In [3]:
def load_text_files(directory):
    files = os.listdir(directory)
    files = [file for file in files if not file.startswith('.DS_Store')]
    files = sorted(files, key=lambda x: int(re.split('\.|_', x)[1]))
    data = []
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8',errors='ignore') as f:
                text = f.read()
                data.append(text)
    return data

In [4]:
# Load text files from directories
original_text = load_text_files('/content/sumpubmed/abstract')
summaries = load_text_files('/content/sumpubmed/shorter_abstract')
# Create a dataset from the loaded text files
data = {'original_text': original_text, 'summary': summaries}

In [5]:
# Create a dataset from the loaded text files
df =pd.DataFrame({'original_text': original_text, 'summary': summaries})

# dataset = Dataset.from_dict(data)

print(df.shape)

(32689, 2)


##Analysis of Bart-large-cnn
It performs extractive summarization. Summary is created from the sentences available in the original text. The sentences created are logically and grammatically correct and has coherence.





In [15]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
returned_summary=summarizer(df['original_text'][4]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][4])
print("\n\nGroundtruth summary:\n"+df['summary'][4])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

Original text:
BACKGROUND
perennial ryegrass  is an important pasture and turf crop. biotechniques such as gene expression studies are being employed to improve traits in this temperate grass. quantitative reverse transcription-polymerase chain reaction  is among the best methods available for determining changes in gene expression. before analysis of target gene expression, it is essential to select an appropriate normalisation strategy to control for non-specific variation between samples. reference genes that have stable expression at different biological and physiological states can be effectively used for normalisation; however, their expression stability must be validated before use.


RESULTS
existing serial analysis of gene expression data were queried to identify six moderately expressed genes that had relatively stable gene expression throughout the year. these six candidate reference genes  were validated for qrt-pcr normalisation in  <dig> diverse perennial ryegrass  sample

##Analysis of t5-base
It performs extractive summarization. Summary is created from the sentences available in the original text. The sentences created are logically and grammatically correct and has coherence.

In [10]:
summarizer = pipeline("summarization", model="t5-base")
returned_summary=summarizer(df['original_text'][4]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][4])
print("\n\nGroundtruth summary:\n"+df['summary'][4])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Original text:
BACKGROUND
perennial ryegrass  is an important pasture and turf crop. biotechniques such as gene expression studies are being employed to improve traits in this temperate grass. quantitative reverse transcription-polymerase chain reaction  is among the best methods available for determining changes in gene expression. before analysis of target gene expression, it is essential to select an appropriate normalisation strategy to control for non-specific variation between samples. reference genes that have stable expression at different biological and physiological states can be effectively used for normalisation; however, their expression stability must be validated before use.


RESULTS
existing serial analysis of gene expression data were queried to identify six moderately expressed genes that had relatively stable gene expression throughout the year. these six candidate reference genes  were validated for qrt-pcr normalisation in  <dig> diverse perennial ryegrass  sample

In [11]:
summarizer = pipeline("summarization", model="t5-base")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

##Analysis of L-macc/autotrain-Biomedical_sc_summ-1217846144
The summary generated makes sense but getting truncated.

In [6]:
summarizer = pipeline("summarization", model="L-macc/autotrain-Biomedical_sc_summ-1217846144")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [525], which does not match the required output shape [1, 525]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 128, 1], which does not match the required output shape [1, 5, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor

Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

##Analysis for L-macc/autotrain-Biomedical_sc_summ-1217846142
The summary generated makes sense but getting truncated.

In [14]:
summarizer = pipeline("summarization", model="L-macc/autotrain-Biomedical_sc_summ-1217846142")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [525], which does not match the required output shape [1, 525]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 128, 1], which does not match the required output shape [1, 5, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor

Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

##Analysis for L-macc/autotrain-Biomedical_sc_summ-1217846148
The summary generated makes sense but getting truncated.

In [8]:
summarizer = pipeline("summarization", model="L-macc/autotrain-Biomedical_sc_summ-1217846148")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [525], which does not match the required output shape [1, 525]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 128, 1], which does not match the required output shape [1, 5, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor

Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

##Analysis for Jacobsith/autotrain-Hello_there-1209845735
The summary generated makes sense but getting truncated.

In [9]:
summarizer = pipeline("summarization", model="Jacobsith/autotrain-Hello_there-1209845735")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

##Analysis for Jacobsith/autotrain-Hello_there-1209845735
The summary generator is copying the same text from the original text into the summary and then getting truncated.

In [10]:
summarizer = pipeline("summarization", model="Blaise-g/led_pubmed_sumpubmed_1")
returned_summary=summarizer(df['original_text'][5]) #do_sample=False)#max_length=130, min_length=30,
print("Original text:\n"+df['original_text'][5])
print("\n\nGroundtruth summary:\n"+df['summary'][5])
print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Your max_length is set to 512, but your input_length is only 426. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=213)


Original text:
BACKGROUND
aspartyl -β-hydroxylase  hydroxylates asp and asn residues within egf-like domains of notch and jagged, which mediate cell motility and differentiation. this study examines the expression, regulation and function of aah, and its related transcripts, humbug and junctin, which lack catalytic domains, using sh-sy5y neuroblastoma cells.


RESULTS
real time quantitative rt-pcr demonstrated 8- or 9-fold higher levels of humbug than aah and junctin, and lower levels of all  <dig> transcripts in normal human brains compared with neuroblastic tumor cells. aah and humbug expression were significantly increased in response to insulin and igf-i stimulation, and these effects were associated with increased directional motility. however, over-expression of aah and not humbug significantly increased motility. treatment with chemical inhibitors of akt, erk mapk, or cyclin-dependent kinase  <dig>  significantly reduced igf-i stimulated aah and humbug expression and motility re

In [9]:
'''
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments

# Example training data
training_data=df[:100].to_dict(orient='records')
print(training_data[0:1])

# Initialize PEGASUS tokenizer and model
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize the data and prepare it for training
tokenized_data = []
for example in training_data:
    source_tokens = tokenizer.encode_plus(
        example["original_text"],
        truncation=True,
        padding="max_length",
        max_length=1024,
        return_tensors="pt"
    )
    target_tokens = tokenizer.encode_plus(
        example["summary"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    tokenized_example = {
        "input_ids": source_tokens.input_ids.flatten(),
        "attention_mask": source_tokens.attention_mask.flatten(),
        "labels": target_tokens.input_ids.flatten(),
    }
    tokenized_data.append(tokenized_example)

# Convert tokenized data to a PyTorch Dataset
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.examples[idx].items()}

train_dataset = CustomDataset(tokenized_data)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
)

# Initialize Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()
'''

[{'original_text': 'BACKGROUND\nthe phytohormone indole-3-acetic acid  is widely distributed among plant-associated bacteria. certain strains of the pseudomonas syringae complex can further metabolize iaa into a less biologically active amino acid conjugate, 3-indole-acetyl-ε-l-lysine, through the action of the iaal gene. in p. syringae and pseudomonas savastanoi strains, the iaal gene is found in synteny with an upstream gene, here called mate, encoding a putative mate family transporter. in p. syringae pv. tomato  dc <dig>  a pathogen of tomato and arabidopsis plants, the hrpl sigma factor controls the expression of a suite of virulence-associated genes via binding to hrp box promoters, including that of the iaal gene. however, the significance of hrpl activation of the iaal gene in the virulence of pto dc <dig> is still unclear.\n\n\nRESULTS\na conserved hrp box motif is found upstream of the iaal gene in the genomes of p. syringae strains. however, although the promoter region of m

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

<ipython-input-9-f5d51c8708a3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.examples[idx].items()}


IndexError: ignored